In [453]:
!ls /tmp/ | grep "ipynb"


16-10-2017 Download vacancies.ipynb
16-10-2017 Text mining.ipynb
2017-03-21_Advanced_Python_Practice_(Nikolay_Markov).ipynb
2017-04-06_Text_mining.ipynb
ExploerData.ipynb
tf-idf.ipynb
Untitled.ipynb


data_dir = 'data'
filename = 'gender_age_dataset.txt'
file_path = '/'.join([data_dir,filename])
url = 'http://data.newprolab.com/data-newprolab-com/project01/' + filename

if not os.path.isdir(data_dir): os.mkdir(data_dir)
if not os.path.isfile(file_path):
    print('Downloading ' + filename + '...')
    urlretrieve(url, file_path)
    print('Download completed')


In [1]:
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier

from tqdm import tqdm_notebook
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn import linear_model
import re
from tqdm import tqdm
import os
import json
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote
import xgboost

import pickle #for save dicts

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Переводим из json в список доменов
def add_domains(pred_test):
    s = pd.Series([])
    pred_test['domains'] = '-'
    print('Starting')
    for ind, a in pred_test['user_json'].iteritems():
        text = []
        for i in json.loads(a)['visits']:
            #print(url2domain(i['url']))
            text.append(url2domain(i['url']))
        s.loc[ind] = ' '.join(text)
        if ind % 361 == 0:
            print('*', end='')
    return s

def url2domain(url):
  try:
      a = urlparse(unquote(url.strip()))
      if (a.scheme in ['http','https']):
          b = re.search("(?:www\.)?(.*)",a.netloc).group(1)
          if b is not None:
              return str(b).strip()
          else:
              return ''
      else:
          return ''
  except:
      return

df = pd.read_csv('/data/home/alexander.chaptykov/data/gender_age_dataset.txt', sep='\t')

train = df[~((df.gender == '-') & (df.age == '-'))]
#создаем серию из доменов
domains_seria = add_domains(train)

In [3]:
#0 Start читаем уже созданный файл с доменами
df = pd.read_csv('/data/home/alexander.chaptykov/data/gender_age_dataset_domains.txt', sep='\t')
df = df.drop(['Unnamed: 0','user_json'], axis=1)
df = df.sort_values(['gender'], ascending=False).reset_index( drop=True)

#Чистим
df['domains'] = df['domains'].map(lambda x: re.sub('https:', '', x))
df['domains'] = df['domains'].map(lambda x: re.sub('http:', '', x))
df['domains'] = df['domains'].map(lambda x: re.sub('[.]', '', x))

#делаем сплит-таргет
df['split_target'] = df['gender'].map(str)+'=='+df['age'].map(str)


In [114]:

#пока не делаем
df['age'].astype('category').cat.codes


0        2
1        1
2        3
3        1
4        2
5        2
6        3
7        2
8        2
9        2
10       2
11       2
12       3
13       2
14       4
15       2
16       2
17       2
18       4
19       2
20       2
21       2
22       3
23       3
24       2
25       3
26       4
27       1
28       2
29       2
        ..
41108    0
41109    0
41110    0
41111    0
41112    0
41113    0
41114    0
41115    0
41116    0
41117    0
41118    0
41119    0
41120    0
41121    0
41122    0
41123    0
41124    0
41125    0
41126    0
41127    0
41128    0
41129    0
41130    0
41131    0
41132    0
41133    0
41134    0
41135    0
41136    0
41137    0
Length: 41138, dtype: int8

# Учеба - возраст

In [1025]:
"""test = ['программировать если хочу пит']
target = ['Progr','rest','help','art']
corpus = [
    'программировать читать думать мечтать создавать творить',
    'играть веселиться петь отдыхать летать плавать',
    'заботиться помощь дарить помогать ростить следить готовить',
    'книги рисовать музицировать муза слушать философствовать ',]
"""
def token_r_without_lemm(text):
     #Токенизатор
    _split = re.compile(r'([^\w]|[+])', re.UNICODE).split
    
    words = [t for t in _split(text.lower()) if t and not t.isspace()]
    
    #возвращем только слова больше 3
    return [word for word in words if len(word) > 3]

def train_model(corpus, target,stp):
    global vocab
    global model
    global cv
    global fit_matrix
    #cv = CountVectorizer( binary = True, ngram_range=(1, 6)) - хорошие параметры для предикта пола
    cv = CountVectorizer( tokenizer=token_r_without_lemm, ngram_range=(1, 4),binary = True, stop_words=stp)
    print('Делаем матрицу...')
    fit_matrix = cv.fit_transform(corpus)
    vocab = cv.get_feature_names()
    x_train, x_test, y_train, y_test = train_test_split(fit_matrix, target, random_state =111 )
    model = LogisticRegression()
    print("Обучение...\n")
    model.fit(x_train, y_train)
    predict = model.predict(x_test)
    print("Точность: %s" % str(accuracy_score(y_test, predict)))
    
def predict_model(test, model, vocab):
    av= CountVectorizer(vocabulary=vocab)
    test = av.fit_transform(test)
    #Предсказание на тестовых данных
    return model.predict(test)

def predict_model_proba(test, model, vocab):
    av= CountVectorizer(vocabulary=vocab)
    test = av.fit_transform(test)
    #Предсказание на тестовых данных
    return model.predict_proba(test)

In [1030]:
###########################
#обучение корпус и таргет
train_model(df[ (df['gender']!='-') & (df['gender'] =='F') ]['domains'], 
            df[(df['gender']!='-') & (df['gender'] =='F')]['age'],
            [x[0] for x in c_words[:10]])

 

Делаем матрицу...
Обучение...

Точность: 0.390366972477


In [1031]:
pred = predict_model(df[df['gender']=='-']['domains'],model, vocab)

df_pred_f_age = pd.DataFrame(pred, columns=['f_age'])


In [1021]:
pred = predict_model(df[df['gender']=='-']['domains'],model, vocab)

df_pred_M_age = pd.DataFrame(pred, columns=['m_age'])


In [1028]:
# возраст мужчин
pred_proba = predict_model_proba(df[df['gender']=='-']['domains'],model, vocab)
df_pred_proba_Man_age = pd.DataFrame(pred_proba.round(3))
df_pred_proba_Man_age.columns = model.classes_

/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [1032]:



# возраст женщин
pred_proba = predict_model_proba(df[df['gender']=='-']['domains'],model, vocab)
df_pred_proba_Female_age = pd.DataFrame(pred_proba.round(3))
df_pred_proba_Female_age.columns = model.classes_
df_pred_proba_Female_age

/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


,18-24,25-34,35-44,45-54,>=55
0,0.150,0.169,0.061,0.612,0.007
1,0.138,0.598,0.126,0.116,0.022
2,0.000,0.997,0.000,0.003,0.000
3,0.197,0.417,0.165,0.184,0.036
4,0.179,0.412,0.208,0.149,0.051
5,0.127,0.409,0.328,0.102,0.033
6,0.059,0.086,0.000,0.713,0.142
7,0.000,0.000,0.000,0.000,1.000
8,0.000,0.000,0.492,0.001,0.508
9,0.015,0.482,0.007,0.494,0.002


# Учеба - пол

In [9]:
"""test = ['программировать если хочу пит']
target = ['Progr','rest','help','art']
corpus = [
    'программировать читать думать мечтать создавать творить',
    'играть веселиться петь отдыхать летать плавать',
    'заботиться помощь дарить помогать ростить следить готовить',
    'книги рисовать музицировать муза слушать философствовать ',]
"""
def token_r_without_lemm(text):
     #Токенизатор
    _split = re.compile(r'([^\w]|[+])', re.UNICODE).split
    
    words = [t for t in _split(text.lower()) if t and not t.isspace()]
    
    #возвращем только слова больше 3
    return [word for word in words if len(word) > 3]

def train_model(corpus, target,stp):
    global vocab
    global model
    global cv
    global fit_matrix
    #cv = CountVectorizer( binary = True, ngram_range=(1, 6)) - хорошие параметры для предикта пола
    cv = CountVectorizer( tokenizer=token_r_without_lemm, binary = True, stop_words=stp)
    print('Делаем матрицу...')
    fit_matrix = cv.fit_transform(corpus)
    vocab = cv.get_feature_names()
    x_train, x_test, y_train, y_test = train_test_split(fit_matrix, target, random_state =111 )
    model = LogisticRegression()
    print("Обучение...\n")
    model.fit(x_train, y_train)
    predict = model.predict(x_test)
    print("Точность: %s" % str(accuracy_score(y_test, predict)))
    
def predict_model(test, model, vocab):
    av= CountVectorizer(vocabulary=vocab)
    test = av.fit_transform(test)
    #Предсказание на тестовых данных
    return model.predict(test)

def predict_model_proba(test, model, vocab):
    av= CountVectorizer(vocabulary=vocab)
    test = av.fit_transform(test)
    #Предсказание на тестовых данных
    return model.predict_proba(test)

In [25]:
#обучение корпус и таргет
train_model(df[(df['gender']!='-')]['domains'], 
            df[(df['gender']!='-')]['gender'],
           [x[0] for x in c_words[:10]])

Делаем матрицу...
Обучение...

Точность: 0.693082457111


In [26]:
#Предсказание
pred = predict_model(df[df['gender']=='-']['domains'],model, vocab)

df_pred = pd.DataFrame(pred)

pred_proba = predict_model_proba(df[df['gender']=='-']['domains'],model, vocab)


/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [76]:
pred_proba

array([[  2.13293830e-01,   7.86706170e-01],
       [  6.72610638e-01,   3.27389362e-01],
       [  9.99999997e-01,   2.69461914e-09],
       ..., 
       [  9.17395863e-01,   8.26041373e-02],
       [  9.99999825e-01,   1.75224020e-07],
       [  1.71502368e-10,   1.00000000e+00]])

In [563]:
df_pred_proba = pd.DataFrame(pred_proba.round(3))
threshold = 0.2
sum(fit_matrix).toarray()[0]
df_pred_proba.columns = model.classes_


In [561]:
#предикт
df_pred_proba = pd.DataFrame(pred_proba.round(3))
threshold = 0.2

df_pred_proba.columns = model.classes_

df_pred_proba_tresh = df_pred_proba[
                (df_pred_proba['F==18-24']>threshold) 
              | (df_pred_proba['F==25-34']>threshold) 
              | (df_pred_proba['F==35-44']>threshold)
              | (df_pred_proba['F==45-54']>threshold)
              | (df_pred_proba['F==>=55']>threshold)
              | (df_pred_proba['M==18-24']>threshold)
              | (df_pred_proba['M==25-34']>threshold)
              | (df_pred_proba['M==35-44']>threshold)
              | (df_pred_proba['M==45-54']>threshold)
              | (df_pred_proba['M==>=55']>threshold)]

df_pred_proba_tresh_indexes = df_pred_proba_tresh.index.values

df_pred_with_tresh = df_pred.iloc[df_pred_proba_tresh_indexes]
print(len(df_pred_with_tresh))
#split!!!!
pd_split = pd.DataFrame(df_pred_with_tresh[0].str.split('==',1).tolist(), columns = ['gender','age'], index = df_pred_proba_tresh_indexes)

df_result = pd.concat([pred_test[['uid']], pd_split], axis=1)


KeyError: 'F==18-24'

In [440]:
final_df = (
    pd
    .concat([pred_test[['uid']], df_result[['gender','age']]], axis=1, join_axes=[pred_test.index])
    .fillna(value='-')
    .sort_values(by='uid'))

In [441]:
final_df.to_csv('/data/home/alexander.chaptykov/project01_gender-age.csv', sep='\t')

In [229]:
## после этого идем в результирующий датафрейм

,18-24,25-34,35-44,45-54,>=55
0,0.001,0.638,0.032,0.300,0.029
1,0.143,0.546,0.250,0.047,0.014
2,0.470,0.178,0.352,0.000,0.000
3,0.111,0.542,0.260,0.066,0.021
4,0.107,0.526,0.241,0.097,0.029
5,0.141,0.368,0.353,0.096,0.042
6,0.277,0.020,0.700,0.001,0.002
7,0.000,0.000,0.333,0.333,0.333
8,0.000,0.329,0.000,0.333,0.338
9,0.000,0.499,0.501,0.000,0.000


In [199]:
#Предсказание
pred = predict_model(pred_test.loc[indexes_of_predicted_man]['domains'],model, vocab)

df_pred = pd.DataFrame(pred)

pred_proba = predict_model_proba(df[df['gender']=='-']['domains'],model, vocab)


/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [210]:
df_pred

,0
0,25-34
1,45-54
2,45-54
3,25-34
4,25-34
5,35-44
6,18-24
7,>=55
8,45-54
9,25-34


In [216]:
df_pred_proba_tresh

,18-24,25-34,35-44,45-54,>=55
0,0.001,0.638,0.032,0.300,0.029
1,0.143,0.546,0.250,0.047,0.014
3,0.111,0.542,0.260,0.066,0.021
4,0.107,0.526,0.241,0.097,0.029
6,0.277,0.020,0.700,0.001,0.002
9,0.000,0.499,0.501,0.000,0.000
10,0.000,0.041,0.000,0.333,0.626
13,0.000,0.812,0.123,0.049,0.015
15,0.123,0.504,0.255,0.095,0.024
16,0.054,0.942,0.002,0.001,0.001


In [561]:
#предикт
df_pred_proba = pd.DataFrame(pred_proba.round(3))
threshold = 0.5

df_pred_proba.columns = model.classes_

df_pred_proba_tresh = df_pred_proba[
                (df_pred_proba['18-24']>threshold) 
              | (df_pred_proba['25-34']>threshold) 
              | (df_pred_proba['35-44']>threshold)
              | (df_pred_proba['45-54']>threshold)
              | (df_pred_proba['>=55']>threshold)]

df_pred_proba_tresh_indexes = df_pred_proba_tresh.index.values

df_pred_with_tresh = df_pred.iloc[df_pred_proba_tresh_indexes]
print(len(df_pred_with_tresh))
#split!!!!
pd_split = pd.DataFrame(df_pred_with_tresh[0].str.split('==',1).tolist(), columns = ['gender','age'], index = df_pred_proba_tresh_indexes)

df_result = pd.concat([pred_test[['uid']], pd_split], axis=1)


KeyError: 'F==18-24'

In [440]:
final_df = (
    pd
    .concat([pred_test[['uid']], df_result[['gender','age']]], axis=1, join_axes=[pred_test.index])
    .fillna(value='-')
    .sort_values(by='uid'))

In [441]:
final_df.to_csv('/data/home/alexander.chaptykov/project01_gender-age.csv', sep='\t')

In [337]:
pred_test = df[df['gender']=='-'].reset_index(drop=True)[['uid','gender','age']]


## Делаем список фрекуенси слов

In [73]:
def ret_sorted(cv, matrix):
    #подсчет частотности
    words = [y[0] for y in sorted(cv.vocabulary_.items(), key = lambda x: x[1])]
    c_words = zip(words, sum(fit_matrix).toarray()[0])
    c_words = sorted(c_words, key=lambda x: x[1],  reverse = True)
    return c_words

In [74]:
c_words = ret_sorted(cv, fit_matrix)

# Учеба - гендер

In [157]:
pred_test.iloc[df_pred_proba_tresh_indexes]

,gender,age,uid,domains,split_target
2,-,-,3efbf44a-2536-4f37-bafa-c93e092107b8,echomskru echomskru echomskru echomskru echoms...,-==-
6,-,-,3ef9b4f6-9720-41a0-9df1-039a4c445a9b,interfaxru mkru newsramblerru topnewsru gomail...,-==-
7,-,-,3eebff7c-a4dd-4eac-aabe-95047b84bab1,autosport37ru gomailru autosport37ru autosport...,-==-
8,-,-,3ef575f4-6d10-4af5-b2b8-5de01d086c88,lovemailru lovemailru lovemailru lovemailru lo...,-==-
9,-,-,3ef569f2-cc9b-440d-8b45-cf6cc6607503,tehnosilaru tehnosilaru tehnosilaru tehnosilar...,-==-
10,-,-,53014866-2869-4405-974a-3d3dc207440b,playcastru playcastru playcastru playcastru pl...,-==-
12,-,-,3ef40673-7556-47da-94ea-5142152c69c6,vashmnenieru vashmnenieru cmgdoubleclicknet cm...,-==-
14,-,-,530b6ff5-63f4-4617-9215-7b6d4144e4b6,koolinarru koolinarru acsconru wikiplanetru al...,-==-
16,-,-,530fb9e9-8ca5-486b-9521-92534ba03fbd,interfax-russiaru 2015godkozycom interfaxru mu...,-==-
17,-,-,53175ada-1a05-4efe-a83b-c420c114a6cb,worldofwarshipsru worldofwarshipsru worldofwar...,-==-


In [165]:
pred_test

,gender,age,uid,domains,split_target
4986,-,-,0000e7ca-32e6-4bef-bdca-e21c025071ff,hotelcosmosru hotelcosmosru business101ru uche...,-==-
4985,-,-,0000f3cf-6e9a-4eab-92f4-cefdad108c83,classesru googlecom zenit-penzaru googlecom ze...,-==-
4984,-,-,000381a6-0400-40f8-98c0-93a6c0852d2d,russianfoodcom russianfoodcom russianfoodcom r...,-==-
4972,-,-,00062d89-a3ba-40dd-9a25-212c9f429133,homewebaltaru homewebaltaru homewebaltaru home...,-==-
4961,-,-,0009233e-a0c2-438e-ac36-bd3e1b7dad1e,i-suxcom i-suxcom fucked-tubecom fucked-tubeco...,-==-
4933,-,-,000c67ae-e4e2-465e-9b22-ce67bbe1385c,shoplenovocom,-==-
4960,-,-,000c737c-5cfc-4bc2-aa20-daf61068a69b,centreoptru centreoptru cssgamebananacom kanji...,-==-
4940,-,-,000de64c-b8b0-4eed-874a-06f685e17920,rubongacamscom ah-mecom ah-mecom ah-mecom ah-m...,-==-
4939,-,-,000e562a-fb49-4d51-a692-787fcec76931,lovemailru lovemailru ebaycom ebaycom catcarin...,-==-
4938,-,-,000fcabd-89eb-4d5a-8ad0-c2bebded6388,datingmetaua,-==-


In [1016]:
#выполняется 1 раз
pred_test = df[df['gender']=='-']
pred_test = pred_test.reset_index(drop=True)


In [1012]:
"""test = ['программировать если хочу пит']
target = ['Progr','rest','help','art']
corpus = [
    'программировать читать думать мечтать создавать творить',
    'играть веселиться петь отдыхать летать плавать',
    'заботиться помощь дарить помогать ростить следить готовить',
    'книги рисовать музицировать муза слушать философствовать ',]
"""
def token_r_without_lemm(text):
     #Токенизатор
    _split = re.compile(r'([^\w]|[+])', re.UNICODE).split
    
    words = [t for t in _split(text.lower()) if t and not t.isspace()]
    
    #возвращем только слова больше 3
    return [word for word in words if len(word) > 3]

def train_model(corpus, target):
    global vocab
    global model
    global cv
    global fit_matrix
    #cv = CountVectorizer( binary = True, ngram_range=(1, 6)) - хорошие параметры для предикта пола
    cv = CountVectorizer( tokenizer=token_r_without_lemm, binary = True)
    print('Делаем матрицу...')
    fit_matrix = cv.fit_transform(corpus)
    vocab = cv.get_feature_names()
    x_train, x_test, y_train, y_test = train_test_split(fit_matrix, target, random_state =111 )
    model = LogisticRegression()
    print("Обучение...\n")
    model.fit(x_train, y_train)
    predict = model.predict(x_test)
    print("Точность: %s" % str(accuracy_score(y_test, predict)))
    
def predict_model(test, model, vocab):
    av= CountVectorizer(vocabulary=vocab)
    test = av.fit_transform(test)
    #Предсказание на тестовых данных
    return model.predict(test)

def predict_model_proba(test, model, vocab):
    av= CountVectorizer(vocabulary=vocab)
    test = av.fit_transform(test)
    #Предсказание на тестовых данных
    return model.predict_proba(test)

In [1013]:
#обучение корпус и таргет
train_model(df[(df['gender']!='-')]['domains'], 
            df[(df['gender']!='-')]['gender'])

Делаем матрицу...
Обучение...

Точность: 0.693082457111


In [1014]:
#Предсказание
pred = predict_model(df[df['gender']=='-']['domains'],model, vocab)

df_pred_gender = pd.DataFrame(pred, columns=['gender'])



In [1015]:
pred_proba = predict_model_proba(df[df['gender']=='-']['domains'],model, vocab)
df_pred_proba = pd.DataFrame(pred_proba.round(3))
df_pred_proba.columns = model.classes_


/usr/local/lib/python3.4/dist-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [901]:
df['age'].value_counts()

25-34    15457
35-44     9360
-         5000
18-24     4898
45-54     4744
>=55      1679
Name: age, dtype: int64

## result_df

In [1033]:
df_pred_proba_Man_age.columns =['m18-24','m25-34','m35-44','m45-54','m>=55']



In [978]:
# Вар1
result_df=pd.concat([pred_test[['uid']],df_pred_proba, df_pred_proba_Man_age,df_pred_proba_Female_age, df_pred_gender,df_pred_M_age, df_pred_f_age], axis=1)
result_df['age'] = '25-34'

#сначала делаем пол
tresh_gender = 0.93

 
index_zalivki =result_df[~((result_df['F']>tresh_gender) | (result_df['M'] > tresh_gender))].index.values
# делаем заливку
result_df.loc[index_zalivki, result_df.columns.isin( ['gender', 'age'] )] = '-'

m_age_tresh = 0.8
f_age_tresh = 0.8

 

#M_age = 
result_df2 =result_df[   ( ((result_df['F']>tresh_gender) & (result_df['18-24'] > f_age_tresh)) 
              | ((result_df['F']>tresh_gender) & (result_df['25-34'] > f_age_tresh) )
          | ((result_df['F']>tresh_gender) & (result_df['35-44'] > f_age_tresh) )
          | ((result_df['F']>tresh_gender) & (result_df['45-54'] > f_age_tresh) )
          | ((result_df['F']>tresh_gender) & (result_df['>=55'] > f_age_tresh) ) 
    
          | ((result_df['M']>tresh_gender) & (result_df['m18-24'] > m_age_tresh)) 
          | ((result_df['M']>tresh_gender) & (result_df['m25-34'] > m_age_tresh) )
          | ((result_df['M']>tresh_gender) & (result_df['m35-44'] > m_age_tresh) )
          | ((result_df['M']>tresh_gender) & (result_df['m45-54'] > m_age_tresh) )
          | ((result_df['M']>tresh_gender) & (result_df['m>=55'] > m_age_tresh) ) )]#[result_df['gender']=='M'][['m_age']]


M_age = result_df2[result_df2['gender'] == 'M']['m_age']
F_age = result_df2[result_df2['gender'] == 'F']['m_age']

result_df.iloc[result_age.index,16] = result_age
result_age = F_age.append(M_age)    
result_age  = result_age.sort_index()

In [1041]:
# Вар2
result_df=pd.concat([pred_test[['uid']],df_pred_proba, df_pred_proba_Man_age,df_pred_proba_Female_age, df_pred_gender,df_pred_M_age, df_pred_f_age], axis=1)
result_df['age'] = '25-34'

#сначала делаем пол
tresh_gender = 0.93

 
index_zalivki =result_df[~((result_df['F']>tresh_gender) | (result_df['M'] > tresh_gender))].index.values
# делаем заливку
result_df.loc[index_zalivki, result_df.columns.isin( ['gender', 'age'] )] = '-'

m_age_tresh = 0.9
f_age_tresh = 0.8

#мужчиины
result_df2 = result_df[(result_df['M']>tresh_gender)] 
result_df2 = result_df2[(result_df2['m18-24'] > m_age_tresh) 
           | (result_df2['m25-34'] > m_age_tresh) 
           | (result_df2['m35-44'] > m_age_tresh) 
           | (result_df2['m45-54'] > m_age_tresh) 
           |(result_df2['m>=55'] > m_age_tresh)]

M_age = result_df2['m_age']

#далее женщины
result_df2 = result_df[(result_df['F']>tresh_gender)] 
result_df2 = result_df2[(result_df2['18-24'] > m_age_tresh) 
           | (result_df2['25-34'] > m_age_tresh) 
           | (result_df2['35-44'] > m_age_tresh) 
           | (result_df2['45-54'] > m_age_tresh) 
           |(result_df2['>=55'] > m_age_tresh)]

F_age = result_df2['f_age']

result_age = M_age#.append(F_age)    
result_age  = result_age.sort_index()

result_df.iloc[result_age.index,16] = result_age


#заменяем предикты распределением

index_delim = result_df[result_df['age']!='-'].index.values
result_df['age'].iloc[index_delim[:int(len(index_delim)*0.3)]] = '35-44'
result_df['age'].iloc[index_delim[int(len(index_delim)*0.3)]:] = '25-34'

#тупиковый путь
result_df.iloc[result_age.index,16] = result_age

In [1042]:
final_df = result_df[['uid', 'gender','age']].sort_values(['uid'])

final_df.to_csv('/data/home/alexander.chaptykov/project01_gender-age.csv', sep='\t')

In [975]:
pd.Series([2,3,3], index=[3,2,1]).sort_index()

1    3
2    3
3    2
dtype: int64

## Рандом форест

In [1087]:
"""test = ['программировать если хочу пит']
target = ['Progr','rest','help','art']
corpus = [
    'программировать читать думать мечтать создавать творить',
    'играть веселиться петь отдыхать летать плавать',
    'заботиться помощь дарить помогать ростить следить готовить',
    'книги рисовать музицировать муза слушать философствовать ',]
"""

def train_model_forest(corpus, target,stp):
    global vocab
    global model
    global cv
    global fit_matrix
     #cv = CountVectorizer( binary = True, ngram_range=(1, 6)) - хорошие параметры для предикта пола
    cv = CountVectorizer( tokenizer=token_r_without_lemm, binary=True, stop_words=stp)
    print('Делаем матрицу...')
    
    fit_matrix = cv.fit_transform(corpus)
    vocab = cv.get_feature_names()
    x_train, x_test, y_train, y_test = train_test_split(fit_matrix, target, random_state =111 )
    model = RandomForestClassifier(n_estimators=1000,max_depth=3, random_state=0, n_jobs=3)
    print("Обучение...\n")
    model.fit(x_train, y_train)
    predict = model.predict(x_test)
    print("Точность: %s" % str(accuracy_score(y_test, predict)))
    
def predict_model(test, model, vocab):
    av= CountVectorizer(vocabulary=vocab)
    test = av.fit_transform(test)
    #Предсказание на тестовых данных
    return model.predict(test)

def predict_model_proba(test, model, vocab):
    av= CountVectorizer(vocabulary=vocab)
    test = av.fit_transform(test)
    #Предсказание на тестовых данных
    return model.predict_proba(test)

In [1088]:
train_model_forest(df[ (df['gender']!='-') & (df['gender'] =='M') ]['domains'], 
            df[(df['gender']!='-') & (df['gender'] =='M')]['age'],
            [x[0] for x in c_words[:10]])



Делаем матрицу...
Обучение...

Точность: 0.462032085561


In [ ]:
train_model(df[(df['gender']!='-')]['domains'], 
            df[(df['gender']!='-')]['gender'],
           [x[0] for x in c_words[:10]])

In [473]:
#обучение корпус и таргет
train_model_forest(df[(df['gender']!='-')]['domains'], 
            df[(df['gender']!='-')]['split_target'], 
            [x[0] for x in c_words[:1]]+[x[0] for x in c_words[-1:]])

Делаем матрицу...
Обучение...

Точность: 0.24128389596


## Буст


In [1099]:
import xgboost as xgb

In [1112]:
def train_model_boost(corpus, target,stp):
    global vocab
    global model
    global cv
    global fit_matrix
    global x_train

    #cv = CountVectorizer( binary = True, ngram_range=(1, 6)) - хорошие параметры для предикта пола
    cv = CountVectorizer(max_features=1000, tokenizer=token_r_without_lemm,  stop_words=stp)
    print('Делаем матрицу...')
    
    fit_matrix = cv.fit_transform(corpus)
    
    vocab = cv.get_feature_names()
    
    x_train, x_test, y_train, y_test = train_test_split(fit_matrix, target, random_state =111 )
    print(type(x_train))

    
    # перевод матрицы в формат xgb
    xg_train = xgb.DMatrix(x_train, label=y_train)
    print('здесь запара...')
    xg_test = xgb.DMatrix(x_test, label=y_test)
    
    # задаем в виде дикта параметры модели
    param = {}
    # use softmax multi-class classification
    param['objective'] = 'multi:softmax'
    # scale weight of positive examples
    param['eta'] = 0.1
    param['max_depth'] = 6
    param['silent'] = 1
    param['nthread'] = 4
    param['num_class'] = 6
    
    #зачем то делаем список из двух туплей но с названиеями
    watchlist = [(xg_train, 'train'), (xg_test, 'test')]
    
    bst = xgb.train(param, xg_train, 26, watchlist)
    print("Обучение...\n")
    pred = bst.predict(xg_test)
    
    
    predict = model.predict(x_test)
    print("Точность: " ,np.sum(pred == test_Y) / test_Y.shape[0])
 

In [1111]:
x_train

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [1113]:
train_model_boost(df[(df['gender']!='-')]['domains'], 
            df[(df['gender']!='-')]['age'], 
            [x[0] for x in c_words[:1]]+[x[0] for x in c_words[-1:]])

Делаем матрицу...
<class 'scipy.sparse.csr.csr_matrix'>


TypeError: a float is required

In [522]:
X = dataset[:,0:8]
Y = dataset[:,8]

In [524]:
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [535]:
X_train = sparse.csr_matrix(X_train)
y_train = sparse.csr_matrix(y_train)

In [536]:
model = XGBClassifier()
model.fit(X_train, y_train)

ValueError: bad input shape (1, 514)

In [526]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [528]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 77.95%
